<a href="https://colab.research.google.com/github/ravichas/AMPL-Tutorial/blob/master/03_AMPL_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test AMPL on Google Drive

# Warning: this is an experimental notebook
AMPL is using Python 3.6.7 while Colab is currently using 3.6.9 listed below so they do not currently match. Your mileage may vary.

# Goals
- Load pre-installed AMPL installation from Google Drive 
- Test AMPL on Colab.
 
# Requirements
- https://github.com/ravichas/AMPL-Tutorial/blob/master/01_make_AMPL_Google_Drive.ipynb
- AMPL.tar.bz2
- ChEMBL Dataset required for testing AMPL will be uploaded from GitHub. ChEMBL dataset is SCN5A (human) target specific IC50 values. The ML method is regression.   

## Mount your Google Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Copy the backed up AMPL installation folder 

In [2]:
# Copy to runtime
!time cp '/content/drive/My Drive/colab/AMPL.tar.bz2' /content


real	0m5.552s
user	0m0.012s
sys	0m1.448s


In [3]:
# Check AMPL.tar.bz2 exists
import os
assert(os.path.isfile('/content/AMPL.tar.bz2'))

In [4]:
# Install AMPL to Colab
# 3 min
import sys

!time tar -jxf AMPL.tar.bz2


real	1m58.952s
user	1m54.528s
sys	0m18.724s


In [9]:
# Load AMPL in this notebook
import sys

site_packages_path = '/content/AMPL/lib/python3.6/site-packages'
if site_packages_path not in sys.path:
  sys.path.insert(1, site_packages_path)
sys.path

['',
 '/content/AMPL/lib/python3.6/site-packages',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

## Let us run a test AMPL run

In [11]:
# Test AMPL
# 90 min
import pandas as pd
import os

import atomsci.ddm.pipeline.compare_models as cmp
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse

/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/content/AMPL/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (

In [17]:
import os
os.mkdir('chembl_activity_models')

**Set the parameters and create the model**

In [18]:
import pandas as pd
import requests
import io
url = 'https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/datasets/ChEMBL26_SCN5A_IC50_human_ml_ready.csv'
download = requests.get(url).content
df = pd.read_csv(url, index_col=0)
# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.iloc[0:5, 0:5]
df.to_csv('ChEMBL26_SCN5A_IC50_human_ml_ready.csv', index=False)

In [19]:
split_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"split_only": "True",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "False",
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [20]:
split_params = parse.wrapper(split_config)

In [21]:
split_model = mp.ModelPipeline(split_params)

In [22]:
split_uuid = split_model.split_dataset()
split_uuid

number of features: 75


2020-09-23 14:11:29,852 Splitting data by scaffold
2020-09-23 14:11:51,940 Dataset split table saved to /content/ChEMBL26_SCN5A_IC50_human_ml_ready_train_valid_test_scaffold_a325a119-038f-4c41-bd81-a07e94628ee8.csv


'a325a119-038f-4c41-bd81-a07e94628ee8'

In [23]:
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


## Train the mode (~ 10 minutes)

In [25]:
train_config = {
"script_dir": "/content/AMPL/atomsci/ddm",
"dataset_key" : "/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv",
"datastore": "False",
"uncertainty": "False",
"splitter": "scaffold",
"split_valid_frac": "0.15",
"split_test_frac": "0.15",
"previously_split": "True",
"split_uuid": "{}".format(split_uuid),
"prediction_type": "regression",
"response_cols" : "pIC50",
"id_col": "compound_id",
"smiles_col" : "base_rdkit_smiles",
"result_dir": "/content/chembl_activity_models",
"system": "LC",
"transformers": "True",
"model_type": "NN",
"featurizer": "graphconv",
"descriptor_type": "graphconv",
"learning_rate": "0.0007",
"layer_sizes": "64,64,32",
"dropouts" : "0.0,0.0,0.0",
"save_results": "False",
"max_epochs": "100",
"verbose": "True"
}

In [26]:
train_params = parse.wrapper(train_config)

In [27]:
train_model = mp.ModelPipeline(train_params)

## Train_model took ~ 30 minutes on a TPU

In [29]:
train_model.train_model()

number of features: 75


2020-09-23 14:14:51,753 Previous dataset split restored


n_cnt [1729.]
y_means [5.15460889]
y_stds [0.64359373]
TIMING: dataset construction took 1.518 s
Loading dataset from disk.
TIMING: dataset construction took 0.250 s
Loading dataset from disk.
TIMING: dataset construction took 0.341 s
Loading dataset from disk.


/content/AMPL/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1/model/model-25


...

INFO:tensorflow:Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1/model/model-2475


2020-09-23 14:38:57,594 Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1/model/model-2475


INFO:tensorflow:Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1/model/model-750


/content/AMPL/lib/python3.6/site-packages/atomsci/ddm/pipeline/model_wrapper.py:767: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  chkpt_dict = yaml.load(chkpt_in.read())
2020-09-23 14:42:08,141 Restoring parameters from /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1/model/model-750
2020-09-23 14:44:54,756 Wrote model metadata to file /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1/model_metadata.json
2020-09-23 14:44:55,008 Wrote model metrics to file /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready/NN_graphconv_scaffold_regression/8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1/model_metrics.json


Wrote model tarball to /content/chembl_activity_models/ChEMBL26_SCN5A_IC50_human_ml_ready_model_8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1.tar.gz


In [30]:
perf_df = cmp.get_filesystem_perf_results('/content/chembl_activity_models', pred_type='regression')
perf_df

Found data for 1 models under /content/chembl_activity_models


,model_uuid,model_type,dataset_key,featurizer,splitter,model_score_type,feature_transform_type,learning_rate,dropouts,layer_sizes,best_epoch,max_epochs,rf_estimators,rf_max_features,rf_max_depth,model_choice_score,train_r2_score,train_rms_score,train_mae_score,valid_r2_score,valid_rms_score,valid_mae_score,test_r2_score,test_rms_score,test_mae_score
0,8fd4192d-10ee-4ddf-85ce-d11ebc39ffb1,NN,/content/ChEMBL26_SCN5A_IC50_human_ml_ready.csv,graphconv,scaffold,r2,normalization,0.0007,"0.00,0.00,0.00","64,64,32",59,100,NaN,NaN,NaN,0.387155,0.782672,0.298918,0.220824,0.387155,0.460054,0.33941,0.253781,0.606358,0.393074
